<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/KISTA_WebUI_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title # Kista webui Colab - v1.3
import requests
from pathlib import Path
from configparser import ConfigParser
from IPython.display import display, Markdown, clear_output
from ipywidgets import widgets, GridspecLayout, Layout

#@markdown <small>**POPULAR NOTEBOOKS AND PROJECTS BY KIM STEINHAUG:**</small><br>
#@markdown <a href="https://github.com/steinhaug/" target="_blank">
#@markdown <img alt="Open Github profile" src="https://img.shields.io/badge/Steinhaug-Github%20profile-blue?logo=github"></a>
#@markdown <a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/KISTA_WebUI_Colab.ipynb?v1.3" target="_blank">
#@markdown <img alt="Open Notebook in Colab" src="https://img.shields.io/badge/Kista--WebUI-Start%20COLAB-blue?logo=googlecolab"></a>
#@markdown <a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/Dreambooth_Colab_edition_for_people_in_a_hurry_fp16.ipynb?1" target="_blank">
#@markdown <img alt="Open Notebook in Colab" src="https://img.shields.io/badge/Dreambooth--for--people--in--a--hurry-Start%20COLAB-blue?logo=googlecolab"></a>
#@markdown <br><small>All Notebooks are maintained by Kim Steinhaug, use my Github for reporting problems or ideas.</small>


connect_drive = True #@param {type:"boolean"}

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU', end="")
elif gpu_info.find('not found') >= 0:
    print('Not connected to a GPU', end="")
else:
    print('GPU Connected', end="")
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print(', your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))



inip = Path("/content/drive/MyDrive")
if not inip.is_dir():
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Drive already mounted, continue.");

def print_markdown(text):
    display(Markdown(text))

def load_url_as_int(url):
    response = requests.get(url)
    content = response.text.strip()
    try:
        number = int(content)
        return number
    except ValueError:
        print("The content of the URL is not a valid integer.")
        return None

VERSION_CURR_NOTEB = 13
VERSION_LATEST_NOTEBurl = "https://raw.githubusercontent.com/steinhaug/stable-diffusion/main/versions/KISTA_WebUI_Colab.txt"
VERSION_LATEST_NOTEB = load_url_as_int(VERSION_LATEST_NOTEBurl)
UPDATE_TXT = '''
# WeeeHOOO! NoteBook Update available!
Update available, your settings are saved so should be eazy peazy updating :D

[![Open in Colab](https://img.shields.io/badge/steinhaug-Open%20in%20Colab-blue?logo=google-colab)](https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/KISTA_WebUI_Colab.ipynb?v1.4)
'''
if VERSION_LATEST_NOTEB is not None:
    if VERSION_LATEST_NOTEB > VERSION_CURR_NOTEB:
        print_markdown(UPDATE_TXT)

GPU Connected, your runtime has 27.3 gigabytes of available RAM

Mounted at /content/drive


In [2]:
# #@title Load and verify settings
#@markdown ![Load your config](https://raw.githubusercontent.com/steinhaug/stable-diffusion/main/assets/kista-load-your-config.jpg)

INI_FILE_LOC = "/content/drive/MyDrive/docs/config_KISTA_WebUI.ini"

import os
import cv2
import shutil
from IPython.display import display, Markdown, clear_output

def convert_images_to_png(directory):
    if os.path.isdir(directory):
        for filename in os.listdir(directory):
            if filename.lower().endswith(('.jpg', '.jpeg', '.bmp', '.gif')):
                image_path = os.path.join(directory, filename)
                png_filename = os.path.splitext(filename)[0] + ".png"
                png_path = os.path.join(directory, png_filename)
                try:
                    img = cv2.imread(image_path)
                    cv2.imwrite(png_path, img, [cv2.IMWRITE_PNG_COMPRESSION, 9])
                    print(f"Converted {filename} to {png_filename}")
                    os.remove(image_path)
                except (IOError, OSError):
                    print(f"Failed to convert {filename}")

def return__isValidDir(directory):
    if os.path.isdir(directory):
        return True
    else:
        return False

def return__isValidFile(file_path):
    if os.path.isfile(file_path):
        return True
    else:
        return False

def do_we_need_to_download_checkpoint():
    if return__isValidFile(path_cpkt.value):
        return False

    if return__isValidDir(dir__models.value) and find_checkpoint_file(dir__models.value) != "":
        return False

    return True

def copy_sources(destination_dir, string_sources):
    sources = [source.strip() for source in string_sources.split(",")]
    for source in sources:
        if os.path.exists(source):
            if os.path.isfile(source):
                shutil.copy2(source, destination_dir)
                print(f"Copied file: {source}")
            elif os.path.isdir(source):
                shutil.copytree(source, os.path.join(destination_dir, os.path.basename(source)))
                print(f"Copied directory: {source}")
            else:
                print(f"Ignored invalid source: {source}")
        else:
            print(f"Ignored non-existent source: {source}")

def copy_files__directory(source_dir, destination_dir):
    file_names = os.listdir(source_dir)

    for file_name in file_names:
        source_path = os.path.join(source_dir, file_name)
        destination_path = os.path.join(destination_dir, file_name)
        shutil.copy2(source_path, destination_path)
        print(f"Copied file: {file_name}")

def find_checkpoint_file(directory):
    if os.path.isdir(directory):
        for filename in os.listdir(directory):
            if filename.endswith(".ckpt") or filename.endswith(".safetensors"):
                return os.path.join(directory, filename)
    return ""

def createFile__ifNotExist(directory, filename_to_look_for, file_contents):
    env_file_path = os.path.join(directory, filename_to_look_for)
    if not os.path.isfile(env_file_path):
        with open(env_file_path, 'w') as env_file:
            env_file.write(file_contents)
            print(f"Created {filename_to_look_for} in {directory}")

def calculate_directory_size(directory):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            total_size += os.path.getsize(filepath)

    return total_size

def return__bytesToMb(size_in_bytes):
    size_in_mb = size_in_bytes / 1024 / 1024
    return f'{int(size_in_mb)} MB'

def display__dirTotalSize(directory):
    print(f'Total: {return__bytesToMb(calculate_directory_size(directory))} for dir {directory}')

def do_dir_contain_controlnet_models(directory):
    if not return__isValidDir(directory):
        return False
    if calculate_directory_size(directory) > 10000000000:
        return True
    else:
        return False

from pathlib import Path
from IPython.display import display, Markdown, clear_output
from ipywidgets import widgets, GridspecLayout, Layout
from configparser import ConfigParser

def strtobool (val):
    """Quick cast tro boolean
    """
    val = val.lower()
    if val in ('y', 'yes', 't', 'true', 'on', '1'):
        return True
    elif val in ('', 'n', 'no', 'f', 'false', 'off', '0'):
        return False
    else:
        raise ValueError("invalid truth value %r" % (val,))

def getValue (savedconf, key, type):
    try:
        val = savedconf[key]
    except KeyError:
        val = ""
    if type == 'bool':
        return strtobool(val)
    else:
        return val

inip = Path("/content/drive/MyDrive")
if not inip.is_dir():
    from google.colab import drive
    drive.mount('/content/drive')
    clear_output()
else:
    print("Drive already mounted, continue.");
    clear_output()

ext_bulk_controllnet = widgets.Checkbox( value=True, description='controlnet, openpose, 3d-open-pose, depth-lib', disabled=False, indent=False )
ext_bulk_one = widgets.Checkbox( value=True, description='roop, adetailer, dynamic-prompts, clip-interrogator, infinite-image-browsing', disabled=False, indent=False )
ext_bulk_two = widgets.Checkbox( value=False, description='deforum, images-browser, huggingface, civitai-browser', disabled=False, indent=False )
ext_bulk_three = widgets.Checkbox( value=False, description='batchlinks, catppuccin, two-shot, aspect-ratio-helper', disabled=False, indent=False )

task__cache_checkpoints = widgets.Checkbox( value=True, description='Cache Stable Diffusion checkpoints in Google Drive', disabled=False, indent=False )
task__cache_controlnet = widgets.Checkbox( value=True, description='Cache all the ControlNet models in Google Drive', disabled=False, indent=False )

path_cpkt = widgets.Text( value="", placeholder='Path to main checkpoint file', disabled=False )

dir__models = widgets.Text( value="", placeholder='Path to models folder', disabled=False )
dir__controlnet = widgets.Text( value="", placeholder='Path to ControlNet models folder', disabled=False )
dir__outdir_samples = widgets.Text( value="", placeholder='Save all generated images path', disabled=False )
dir__outdir_grids = widgets.Text( value="", placeholder='Save all generated grids path', disabled=False )
dir__extra_loras_search = widgets.Text( value="", placeholder='Extra loras search paths, seperated with comma', disabled=False )

sources__copy_textual_inversion = widgets.Text( value="", placeholder='Textual Inversion files/folders to copy into WebUI. Seperate with ,', disabled=False )
sources__copy_lauras = widgets.Text( value="", placeholder='LORA files/folders to copy into WebUI. Seperate with ,', disabled=False )

def update_KISTA_config():
    config_object = ConfigParser()
    config_object["CONF"] = {
        "ext_bulk_controllnet": ext_bulk_controllnet.value,
        "ext_bulk_one": ext_bulk_one.value,
        "ext_bulk_two": ext_bulk_two.value,
        "ext_bulk_three": ext_bulk_three.value,

        "task__cache_checkpoints": task__cache_checkpoints.value,
        "task__cache_controlnet": task__cache_controlnet.value,

        "path_cpkt": path_cpkt.value,
        "dir__models": dir__models.value,
        "dir__controlnet": dir__controlnet.value,
        "dir__outdir_samples": dir__outdir_samples.value,
        "dir__outdir_grids": dir__outdir_grids.value,
        "dir__extra_loras_search": dir__extra_loras_search.value,
        "sources__copy_textual_inversion": sources__copy_textual_inversion.value,
        "sources__copy_lauras": sources__copy_lauras.value,
    }
    with open(INI_FILE_LOC, 'w') as conf:
        config_object.write(conf)

def btn_continue_event_handler(btn_object):
    grid.close();
    print('You pressed {}'.format(btn_object.description))
    print('Creating .ini')
    update_KISTA_config()
    clear_output()
    print('Your settings is saved and confirmed! ✓')

# - - - - - -

btn_continue = widgets.Button(description='Save configuration and continue', button_style='info', layout=Layout(width='auto', height='auto'))
btn_continue.on_click(btn_continue_event_handler)

btn1 = widgets.Button(description='EXTENSIONS TO LOAD', layout=Layout(width='auto', height='auto'))
btn1.style.button_color='White'; btn1.style.font_weight='bold';
btn2 = widgets.Button(description='GOOGLE DRIVE PATHS', layout=Layout(width='auto', height='auto'))
btn2.style.button_color='White'; btn2.style.font_weight='bold';
btn3 = widgets.Button(description='KISTA TASKS', layout=Layout(width='auto', height='auto'))
btn3.style.button_color='White'; btn3.style.font_weight='bold';
btn4 = widgets.Button(description='MODELS', layout=Layout(width='auto', height='auto'))
btn4.style.button_color='White'; btn4.style.font_weight='bold';

grid = GridspecLayout(8, 3)

grid[0, 0] = btn1
grid[1, 0] = ext_bulk_controllnet
grid[2, 0] = ext_bulk_one
grid[3, 0] = ext_bulk_two
grid[4, 0] = ext_bulk_three

grid[0, 1] = btn3
grid[1, 1] = task__cache_checkpoints
grid[2, 1] = task__cache_controlnet
grid[5, 1] = sources__copy_textual_inversion
grid[6, 1] = sources__copy_lauras

grid[0, 2] = btn2
grid[1, 2] = path_cpkt
grid[2, 2] = dir__models
grid[3, 2] = dir__controlnet
grid[4, 2] = dir__outdir_samples
grid[5, 2] = dir__outdir_grids
grid[6, 2] = dir__extra_loras_search

grid[7, :] = btn_continue

display(grid)

inip = Path(INI_FILE_LOC)
if inip.is_file():
    # Load values
    config_object = ConfigParser()
    config_object.read(INI_FILE_LOC)
    savedconf = config_object["CONF"]
    ext_bulk_controllnet.value = getValue(savedconf, 'ext_bulk_controllnet', 'bool')
    ext_bulk_one.value = getValue(savedconf, 'ext_bulk_one', 'bool')
    ext_bulk_two.value = getValue(savedconf, 'ext_bulk_two', 'bool')
    ext_bulk_three.value = getValue(savedconf, 'ext_bulk_three', 'bool')
    task__cache_checkpoints.value = getValue(savedconf, 'task__cache_checkpoints', 'bool')
    task__cache_controlnet.value = getValue(savedconf, 'task__cache_controlnet', 'bool')
    path_cpkt.value = getValue(savedconf, 'path_cpkt', 'string')
    dir__models.value = getValue(savedconf, 'dir__models', 'string')
    dir__controlnet.value = getValue(savedconf, 'dir__controlnet', 'string')
    dir__outdir_samples.value = getValue(savedconf, 'dir__outdir_samples', 'string')
    dir__outdir_grids.value = getValue(savedconf, 'dir__outdir_grids', 'string')
    dir__extra_loras_search.value = getValue(savedconf, 'dir__extra_loras_search', 'string')
    sources__copy_textual_inversion.value = getValue(savedconf, 'sources__copy_textual_inversion', 'string')
    sources__copy_lauras.value = getValue(savedconf, 'sources__copy_lauras', 'string')
else:
    # Make sure the ./docs folder exists in drive
    if not return__isValidDir('/content/drive/MyDrive/docs'):
        !mkdir /content/drive/MyDrive/docs

# https://tutswiki.com/read-write-config-files-in-python/


Your settings is saved and confirmed! ✓


In [ ]:
# #@title Installer and launcher!
#@markdown ![Load your config](https://raw.githubusercontent.com/steinhaug/stable-diffusion/main/assets/kista-installer-webui-launcher.jpg)

#@markdown Updated version!

skip_install = False #@param {type:"boolean"}

#@markdown <small>_If you need to restart the WebUI check the **skip_install** as you do not need to reinstall anything, loads in ca. 60 seconds_</small><br>
#@markdown <small>_First time install with default settings ca. 13,5 min, next time when using Google Drive all ready in less than 10 min._</small><br>
#@markdown <small>_Rebooting with skip_install: ca. 1 min_</small><br>

import tarfile; import os; import shutil; import json;
from pathlib import Path
from ipywidgets import widgets, GridspecLayout, Layout
from configparser import ConfigParser
from IPython.display import display, Markdown, clear_output
from tqdm import tqdm

#
# 1/8 : DEFINING FUNCTIONS
#
_B=True
_A=False
def return__isValidDir(directory):
    if os.path.isdir(directory):return _B
    else:return _A
def return__isValidFile(file_path):
    if os.path.isfile(file_path):return _B
    else:return _A
def do_we_need_to_download_checkpoint():
    if return__isValidFile(path_cpkt.value):return _A
    if return__isValidDir(dir__models.value) and find_checkpoint_file(dir__models.value)!='':return _A
    return _B
def copy_sources(destination_dir, string_sources):
    sources=[source.strip()for source in string_sources.split(',')]
    for source in sources:
        if os.path.exists(source):
            if os.path.isfile(source):shutil.copy2(source,destination_dir);print(f"Copied file: {source}")
            elif os.path.isdir(source):shutil.copytree(source,os.path.join(destination_dir,os.path.basename(source)));print(f"Copied directory: {source}")
            else:print(f"Ignored invalid source: {source}")
        else:print(f"Ignored non-existent source: {source}")
def copy_files__directory(source_dir, destination_dir):
    file_names = os.listdir(source_dir)
    for file_name in file_names:
        source_path = os.path.join(source_dir, file_name)
        destination_path = os.path.join(destination_dir, file_name)
        shutil.copy2(source_path, destination_path)
        print(f"Copied file: {file_name}")
def find_checkpoint_file(directory):
    if os.path.isdir(directory):
        for filename in os.listdir(directory):
            if filename.endswith(".ckpt") or filename.endswith(".safetensors"):
                return os.path.join(directory, filename)
    return ""
def createFile__ifNotExist(directory, filename_to_look_for, file_contents):
    env_file_path = os.path.join(directory, filename_to_look_for)
    if not os.path.isfile(env_file_path):
        with open(env_file_path, 'w') as env_file:
            env_file.write(file_contents)
            print(f"Created {filename_to_look_for} in {directory}")
def calculate_directory_size(directory):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            total_size += os.path.getsize(filepath)
    return total_size
def return__bytesToMb(size_in_bytes):
    size_in_mb = size_in_bytes / 1024 / 1024
    return f'{int(size_in_mb)} MB'
def display__dirTotalSize(directory):
    print(f'Total: {return__bytesToMb(calculate_directory_size(directory))} for dir {directory}')
def do_dir_contain_controlnet_models(directory):
    if not return__isValidDir(directory):return _A
    if calculate_directory_size(directory)>10000000000:return _B
    else:return _A
def strtobool (val):
    val=val.lower()
    if val in('y','yes','t','true','on','1'):return _B
    elif val in('','n','no','f','false','off','0'):return _A
    else:raise ValueError('invalid truth value %r'%(val,))
def getValue (savedconf, key, type):
    try:val=savedconf[key]
    except KeyError:val=''
    if type=='bool':return strtobool(val)
    else:return val
def openReplaceStringMatch(file_path, match, replace):
    if not os.path.isfile( file_path ):
        print(f'Error opening file: {file_path}')
    else:
        modified_lines = []
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                if match in line:
                    line = line.replace(match, replace)
                modified_lines.append(line)
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(''.join(modified_lines))
def commentOutLinesStartingWith(file_path, matches, comment_str='# '):
    if not os.path.isfile( file_path ):
        print(f'Error opening file: {file_path}')
    else:
        modified_lines = []
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                line = line
                uncomment = False
                for match in matches:
                    if line.strip().startswith(match):
                        print(f'Match: {match}')
                        uncomment = True
                if uncomment:
                    line = comment_str + line.strip() + "\n"
                modified_lines.append(line)
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(''.join(modified_lines))

def move_single_file(source_file, destination_file, copyOnly=False):
    file_size = os.path.getsize(source_file)

    with open(source_file, "rb") as src_file, \
         open(destination_file, "wb") as dest_file, \
         tqdm(total=file_size, unit="B", unit_scale=True, desc="Moving file", ncols=80) as progress:

        chunk_size = 1024 * 1024  # 1 MB
        bytes_copied = 0

        while True:
            chunk = src_file.read(chunk_size)

            if not chunk:
                break

            dest_file.write(chunk)
            bytes_copied += len(chunk)
            progress.update(len(chunk))

    if not copyOnly:
        os.remove(source_file)

    print(f"File '{source_file}' moved to '{destination_file}'.")
def download_all_control_models():
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11e_sd15_ip2p_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11e_sd15_shuffle_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_canny_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11f1p_sd15_depth_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_inpaint_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_lineart_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_mlsd_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_normalbae_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_openpose_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_scribble_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_seg_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_softedge_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15s2_lineart_anime_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11f1e_sd15_tile_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_ip2p_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11e_sd15_ip2p_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_shuffle_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11e_sd15_shuffle_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_canny_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_canny_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1p_sd15_depth_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11f1p_sd15_depth_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_inpaint_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_inpaint_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_lineart_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_lineart_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_mlsd_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_mlsd_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_normalbae_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_normalbae_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_openpose_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_openpose_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_scribble_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_scribble_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_seg_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_seg_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_softedge_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_softedge_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15s2_lineart_anime_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15s2_lineart_anime_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11f1e_sd15_tile_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_style_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_style_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_sketch_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_sketch_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_seg_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_seg_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_openpose_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_openpose_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_keypose_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_keypose_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_depth_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_depth_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_color_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_color_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_canny_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_canny_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_canny_sd15v2.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_canny_sd15v2.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_depth_sd15v2.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_depth_sd15v2.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_sketch_sd15v2.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_sketch_sd15v2.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_zoedepth_sd15v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_zoedepth_sd15v1.pth
    return 1


if not skip_install:
    %cd /content
    print(f'Installing libraries and software...')
    %env TF_CPP_MIN_LOG_LEVEL=1
    !apt -y update -qq
    !wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
    %env LD_PRELOAD=/content/libtcmalloc_minimal.so.4
    !apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev
    !pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118 -U
    !pip install -q xformers==0.0.20 triton==2.0.0 gradio_client==0.2.7 -U
    clear_output();
    print(f'Done!')


#
# 3/8 : DEFINING PATHS
#
DEFAULT_CHECKPOINTS_FOLDER = '/content/drive/MyDrive/models-checkpoints'
DEFAULT_CONTROLNET_FOLDER  = '/content/drive/MyDrive/models-ControlNet-1.1'
WEBUI_ROOT = '/content'
WEBUI_FOLDER = 'stable-diffusion-webui'
WEBUI_PATH = os.path.join(WEBUI_ROOT, WEBUI_FOLDER)

CLONE_ARG = "clone --depth 1"


#
# 4/8 : Cloning repositories
#
if not skip_install:
    %cd $WEBUI_ROOT
    clear_output();
    print(f'Installing WebUI and Extensions...')
    !git clone -b v2.5 https://github.com/camenduru/stable-diffusion-webui {WEBUI_PATH}
    #!git clone https://github.com/camenduru/stable-diffusion-webui

    !git {CLONE_ARG} https://huggingface.co/embed/negative {WEBUI_PATH}/embeddings/negative
    !git {CLONE_ARG} https://huggingface.co/embed/lora {WEBUI_PATH}/models/Lora/positive
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d {WEBUI_PATH}/models/ESRGAN -o 4x-UltraSharp.pth
    !wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O {WEBUI_PATH}/scripts/run_n_times.py
    !git {CLONE_ARG} https://github.com/kohya-ss/sd-webui-additional-networks {WEBUI_PATH}/extensions/sd-webui-additional-networks
    #!git {CLONE_ARG} https://github.com/hnmr293/posex {WEBUI_PATH}/extensions/posex

    hashes_json_bck = '/content/drive/MyDrive/docs/config_KISTA_WebUI.hashes.json'
    hashes_json_org = '/content/stable-diffusion-webui/extensions/sd-webui-additional-networks/hashes.json'
    if return__isValidFile(hashes_json_bck):
        if return__isValidFile(hashes_json_org):
            os.remove(hashes_json_org)
        move_single_file(hashes_json_bck, hashes_json_org,True)

    !git {CLONE_ARG} https://github.com/camenduru/sd-webui-tunnels {WEBUI_PATH}/extensions/sd-webui-tunnels
    !git {CLONE_ARG} https://github.com/camenduru/a1111-sd-webui-locon {WEBUI_PATH}/extensions/a1111-sd-webui-locon
    !git {CLONE_ARG} https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg {WEBUI_PATH}/extensions/stable-diffusion-webui-rembg
    #!git {CLONE_ARG} https://github.com/tjm35/asymmetric-tiling-sd-webui {WEBUI_PATH}/extensions/asymmetric-tiling-sd-webui

    if ext_bulk_controllnet.value:
        !git {CLONE_ARG} https://github.com/Mikubill/sd-webui-controlnet {WEBUI_PATH}/extensions/sd-webui-controlnet
        !git {CLONE_ARG} https://github.com/huchenlei/sd-webui-openpose-editor {WEBUI_PATH}/extensions/sd-webui-openpose-editor
        !git {CLONE_ARG} https://github.com/fkunn1326/openpose-editor {WEBUI_PATH}/extensions/openpose-editor
        !git {CLONE_ARG} https://github.com/jexom/sd-webui-depth-lib {WEBUI_PATH}/extensions/sd-webui-depth-lib
        !git {CLONE_ARG} https://github.com/nonnonstop/sd-webui-3d-open-pose-editor {WEBUI_PATH}/extensions/sd-webui-3d-open-pose-editor

    if ext_bulk_one.value:
        !git {CLONE_ARG} https://github.com/s0md3v/sd-webui-roop  {WEBUI_PATH}/extensions/sd-webui-roop
        #!git {CLONE_ARG} https://github.com/Gourieff/sd-webui-roop-nsfw {WEBUI_PATH}/extensions/sd-webui-roop-nsfw
        !git {CLONE_ARG} https://github.com/Bing-su/adetailer {WEBUI_PATH}/extensions/adetailer
        !git {CLONE_ARG} https://github.com/zanllp/sd-webui-infinite-image-browsing {WEBUI_PATH}/extensions/sd-webui-infinite-image-browsing
        !git {CLONE_ARG} https://github.com/adieyal/sd-dynamic-prompts.git  {WEBUI_PATH}/extensions/sd-dynamic-prompts
        !git {CLONE_ARG} https://github.com/pharmapsychotic/clip-interrogator-ext.git {WEBUI_PATH}/extensions/clip-interrogator-ext

    if ext_bulk_two.value:
        !git {CLONE_ARG} https://github.com/deforum-art/deforum-for-automatic1111-webui {WEBUI_PATH}/extensions/deforum-for-automatic1111-webui
        !git {CLONE_ARG} https://github.com/camenduru/stable-diffusion-webui-images-browser {WEBUI_PATH}/extensions/stable-diffusion-webui-images-browser
        !git {CLONE_ARG} https://github.com/camenduru/stable-diffusion-webui-huggingface {WEBUI_PATH}/extensions/stable-diffusion-webui-huggingface
        !git {CLONE_ARG} https://github.com/camenduru/sd-civitai-browser {WEBUI_PATH}/extensions/sd-civitai-browser

    if ext_bulk_three.value:
        !git {CLONE_ARG} https://github.com/etherealxx/batchlinks-webui {WEBUI_PATH}/extensions/batchlinks-webui
        !git {CLONE_ARG} https://github.com/camenduru/stable-diffusion-webui-catppuccin {WEBUI_PATH}/extensions/stable-diffusion-webui-catppuccin
        !git {CLONE_ARG} https://github.com/ashen-sensored/stable-diffusion-webui-two-shot {WEBUI_PATH}/extensions/stable-diffusion-webui-two-shot
        !git {CLONE_ARG} https://github.com/thomasasfk/sd-webui-aspect-ratio-helper {WEBUI_PATH}/extensions/sd-webui-aspect-ratio-helper

    #!git {CLONE_ARG} https://github.com/AlUlkesh/stable-diffusion-webui-images-browser {WEBUI_PATH}/extensions/stable-diffusion-webui-images-browser
    #!git {CLONE_ARG} https://github.com/yownas/shift-attention {WEBUI_PATH}/extensions/shift-attention
    #!git {CLONE_ARG} https://github.com/canisminor1990/sd-webui-lobe-theme {WEBUI_PATH}/extensions/sd-webui-lobe-theme
    !git {CLONE_ARG} https://github.com/Nevysha/Cozy-Nest  {WEBUI_PATH}/extensions/Cozy-Nest

    %cd {WEBUI_PATH}

    !git reset --hard
    !git -C {WEBUI_PATH}/repositories/stable-diffusion-stability-ai reset --hard


    #!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' {WEBUI_PATH}/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' {WEBUI_PATH}/launch.py
    #!sed -i -e 's/\"sd_model_checkpoint\"\,/\"sd_model_checkpoint\,sd_vae\,CLIP_stop_at_last_layers\"\,/g' {WEBUI_PATH}/modules/shared.py

    !sed -i -e '''/from modules import launch_utils/a\import os''' {WEBUI_PATH}/launch.py
    !sed -i -e '''/        prepare_environment()/a\        os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' {WEBUI_PATH}/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' {WEBUI_PATH}/launch.py
    !sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' {WEBUI_PATH}/modules/shared.py

    clear_output();
    print('Completed installing WebUI and Extensions...')

#
# 5/8 : ControlNet
#
if not skip_install:
    print('Model control, check for Google Drive caches...')
    if ext_bulk_controllnet.value and do_dir_contain_controlnet_models(dir__controlnet.value):
        print('ControlNet is cached with Google Drive')
    elif ext_bulk_controllnet.value:
        print('Downloading ControlNet models...')
        download_all_control_models()
        clear_output()
        if task__cache_controlnet.value:
            if not return__isValidDir(dir__controlnet.value):
                print(f'Creating new drive folder: ./{os.path.basename(DEFAULT_CONTROLNET_FOLDER)}')
                os.makedirs(DEFAULT_CONTROLNET_FOLDER, exist_ok=True)
                dir__controlnet.value = DEFAULT_CONTROLNET_FOLDER
            print('Copying all models onto your Drive folder...')
            copy_files__directory(f'{WEBUI_PATH}/extensions/sd-webui-controlnet/models', dir__controlnet.value)
            task__cache_controlnet.value = False
            update_KISTA_config()
            clear_output()
            print('Configuration updated with new settings... continuing...')
    else:
        clear_output()
        print('Continuing...')

    if sources__copy_lauras.value != "":
        print('Copying LORA files into model folder...')
        copy_sources(f'{WEBUI_PATH}/models/Lora', sources__copy_lauras.value)
        clear_output();
        print('Done.')

    if sources__copy_textual_inversion.value != "":
        print('Copying TEXTUAL_INVERSION files into model folder...')
        copy_sources(f'{WEBUI_PATH}/embeddings', sources__copy_textual_inversion.value)
        clear_output();
        print('Done.')

#
# 6/8 : Checkpoint
#
if do_we_need_to_download_checkpoint():
    print('Downloading uberRealisticPornMerge_urpmv13')
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/urpm/resolve/main/uberRealisticPornMerge_urpmv13.safetensors -d {WEBUI_PATH}/models/Stable-diffusion -o uberRealisticPornMerge_urpmv13.safetensors
    if task__cache_checkpoints.value:
        if not return__isValidDir(dir__models.value):
            dir__models.value = DEFAULT_CHECKPOINTS_FOLDER
            os.makedirs(dir__models.value, exist_ok=True)

        clear_output()
        print(f'Copying checkpoint to Google Drive folder...')
        path_cpkt.value = dir__models.value + '/uberRealisticPornMerge_urpmv13.safetensors'
        shutil.copy2(f'{WEBUI_PATH}/models/Stable-diffusion/uberRealisticPornMerge_urpmv13.safetensors', dir__models.value)
        task__cache_checkpoints.value = False
        update_KISTA_config()
        print(f'Copy complete, KISTA settings updated...')

#
# 7/8 : Post installer checks
#
%cd $WEBUI_PATH
clear_output()
EnvFileData = '''
IIB_SECRET_KEY=
IIB_SERVER_LANG=en
IIB_ACCESS_CONTROL=enable
'''
if not skip_install:
    print('Post installer checks, before starting up launcher...')
    createFile__ifNotExist(f'{WEBUI_PATH}/extensions/sd-webui-infinite-image-browsing', '.env', EnvFileData)
    print('Patching extension: infinite-image-browsing...')
    print(' -- setting up .env file')
    print(' -- correcting .db path')
    openReplaceStringMatch(f'{WEBUI_PATH}/extensions/sd-webui-infinite-image-browsing/scripts/iib/db/datamodel.py','"iib.db"','"/content/drive/MyDrive/docs/config_KISTA_WebUI.db"')
    print()

    print('Patching extension: roop')
    print(' -- disabling naked checker')
    commentOutLinesStartingWith(f'{WEBUI_PATH}/extensions/sd-webui-roop/scripts/cimage.py', ['chunks =','for chunk','shapes.append'],'    # ')
    print()

#
# build settings always
#
inip = Path(f'{WEBUI_PATH}/config.json')
if not skip_install:
    if not inip.is_file():
        print('Preparing WebUI config.json with KISTA settings')
        import json
        conf_ini_json = {
            "outdir_samples": dir__outdir_samples.value,
            "outdir_grids": dir__outdir_grids.value,
            "control_net_models_path": dir__controlnet.value,
            "quicksettings": "sd_model_checkpoint,sd_vae,CLIP_stop_at_last_layers,initial_noise_multiplier,img2img_color_correction",
            "save_txt": True,
            "do_not_add_watermark": True,
            "control_net_no_detectmap": True,
            "hide_samplers": [
                "LMS",
                "Heun",
                "DPM2",
                "DPM2 a",
                "DPM fast",
                "DPM adaptive",
                "LMS Karras",
                "PLMS",
                "DPM++ 2S a Karras",
                "DPM2 a Karras",
                "DPM2 Karras"
            ],
            "additional_networks_hash_thread_count": 4.0,
            "additional_networks_back_up_model_when_saving": False,
            "unload_models_when_training": True,
            "training_xattention_optimizations": True,
            "additional_networks_extra_lora_path": dir__extra_loras_search.value
        }
        with open("config.json", "w") as jsonfile:
            json.dump(conf_ini_json, jsonfile)
    else:
        print('WebUI config.json already exists, all done.')

#
# 8/8: Run WebUI
#
%cd $WEBUI_PATH
clear_output()

# !python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple

args = "--share --xformers --enable-insecure-extension-access"
#if skip_install:
#    args = args + " --skip-install"
#if run_from_drive:
#    args = args + " --skip-torch-cuda-test"
if len(path_cpkt.value) and os.path.isfile(path_cpkt.value):
    ckpt_file = path_cpkt.value
else:
    ckpt_file = ''
if dir__models.value != "":
    args = args + " --ckpt-dir=\"" + dir__models.value + "\""
    if ckpt_file == '':
        ckpt_file = checkpoint_file = find_checkpoint_file(dir__models.value)
if ckpt_file != "":
    args = args + " --ckpt=\"" + ckpt_file + "\""
else:
    print('NOTICE! Default --ckpt invalid, skipped parameter.')

print('Launching WebUI...')

!python launch.py $args


Launching WebUI...
fatal: No names found, cannot describe anything.
Python 3.10.6 (main, May 29 2023, 11:10:38) [GCC 11.3.0]
Version: 1.5.0
Commit hash: a3ddf464a2ed24c999f67ddfef7969f8291567be
Installing gfpgan
Installing clip
Installing open_clip
Cloning Stable Diffusion into /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai...
Cloning into '/content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai'...
remote: Enumerating objects: 574, done.
remote: Counting objects: 100% (304/304), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 574 (delta 244), reused 216 (delta 216), pack-reused 270
Receiving objects: 100% (574/574), 73.43 MiB | 17.16 MiB/s, done.
Resolving deltas: 100% (276/276), done.
Cloning Stable Diffusion XL into /content/stable-diffusion-webui/repositories/generative-models...
Cloning into '/content/stable-diffusion-webui/repositories/generative-models'...
remote: Enumerating objects: 435, done.
remote: Counting

## Tools when not running WebUI

In [ ]:
#@title CHECKPOINT DOWNLOADER: Save in drive models folder
#@markdown *Link to checkpoint/safetensors file*
ModelUrl = "" #@param {type:"string"}

if len(dir__models.value) and len(ModelUrl):
    %cd {dir__models.value}
    !wget --content-disposition {ModelUrl}
else:
    print('error, no folder defined in your config')

In [ ]:
#@title CHECKPOINT DOWNLOADER: Manual save
#@markdown *Link to checkpoint/safetensors file*
ModelUrl = "https://civitai.com/api/download/models/86698" #@param {type:"string"}
ModelSaveDir = "/content/drive/MyDrive/KISTA_WebUI/sd-webui/models/Stable-diffusion/NSFW" #@param ["/content/stable-diffusion-webui/models/Stable-diffusion/NSFW", "/content/drive/MyDrive/KISTA_WebUI/sd-webui/models/Stable-diffusion/NSFW", "/content/drive/MyDrive/KISTA_WebUI/sd-webui/models/VAE"] {allow-input: true}

def return__isValidDir(directory):
    if os.path.isdir(directory):
        return True
    else:
        return False


if len(ModelSaveDir) and len(ModelUrl):
    if return__isValidDir(ModelSaveDir):
        %cd {ModelSaveDir}
        !wget --content-disposition {ModelUrl}
    else:
        print('Directory does not exist!')
else:
    print('error... check inputs...')

In [ ]:
#@title CIVITAI LORA DOWNLOADER: Downloading multiple LORAs by model_id
#@markdown *CivitAI.com API Bearer kode*
bearer_token = '' #@param {type:"string"}
#@markdown *Comma seperated list of Laura model_id's from Civitai.com*<br>
#@markdown <small>Link example: https:<span>//</span>civitai.com/models/**9950**/realistic-vaginas-god-pussy-1</small>
model_ids = '' #@param {type:"string"}
#@markdown *Download destination, see dropown for defaults*
temp_dir = "/content/stable-diffusion-webui/models/Lora/download" #@param ["/content/stable-diffusion-webui/models/Lora/download", "/content/drive/MyDrive/KISTA_WebUI/sd-webui/models/Lora/download"]
cache_dir = "/content/_kista_cache_files"

import hashlib
import os
import cv2
import json
import requests
import subprocess
import shutil
from IPython.display import display, Markdown, clear_output

def convert_images_to_png(directory):
    if os.path.isdir(directory):
        for filename in os.listdir(directory):
            if filename.lower().endswith(('.jpg', '.jpeg', '.bmp', '.gif')):
                image_path = os.path.join(directory, filename)
                png_filename = os.path.splitext(filename)[0] + ".png"
                png_path = os.path.join(directory, png_filename)
                try:
                    img = cv2.imread(image_path)
                    cv2.imwrite(png_path, img, [cv2.IMWRITE_PNG_COMPRESSION, 9])
                    print(f"Converted {filename} to {png_filename}")
                    os.remove(image_path)
                except (IOError, OSError):
                    print(f"Failed to convert {filename}")

def verbose_print(item, indent=0):
    if isinstance(item, dict):
        for key, value in item.items():
            if isinstance(value, dict) or isinstance(value, list):
                verbose_print(value, indent + 2)
            else:
                print(' ' * (indent + 2) + f'{key}: {value}')
    elif isinstance(item, list):
        for element in item:
            if isinstance(element, dict) or isinstance(element, list):
                verbose_print(element, indent + 2)
            else:
                print(' ' * indent + f'Value: {element}')

def curl_request(url):
    curl_output = subprocess.check_output(['curl', url])
    response = curl_output.decode('utf-8')
    return response

def curl_request2(url, cache_dir, bearer_token):
    cache_file = os.path.join(cache_dir, hashlib.md5(url.encode()).hexdigest())
    if os.path.exists(cache_file):
        with open(cache_file, 'r') as f:
            source_code = f.read()
    else:
        curl_command = ['curl', '-H', f'Authorization: Bearer {bearer_token}', url]
        curl_output = subprocess.check_output(curl_command)
        source_code = curl_output.decode('utf-8')
    return source_code

def parse_json_for_model(data):
    for key, value in data.items():
        if key == 'modelVersions':
            print(' ' * (1 + 1) + f'{key}: ')
            for data2 in value:
                for key2, value2 in data2.items():
                    if key2 == 'files':
                        print(' ' * (5 + 1) + f'{key2}:')
                        for data3 in value2:
                            for key3, value3 in data3.items():
                                print(' ' * (9 + 1) + f'{key3}: {value3}')
                    else:
                        print(' ' * (5 + 1) + f'{key2}: {value2}')
        else:
            print(' ' * (1 + 1) + f'{key}: {value}')

def retrieveDownloadData(model_id, bearer_token, cache_dir, verbose_var = False):

    url = f'https://civitai.com/api/v1/images?nsfw=X&modelId={model_id}&limit=1'
    response_data = curl_request2(url, cache_dir, bearer_token)

    data = json.loads(response_data)
    try:
        if 'items' in data and len(data['items']) > 0 and 'url' in data['items'][0]:
            image_url = data['items'][0]['url']
        else:
            image_url = ''
    except (KeyError, IndexError):
        pass

    url = f'https://civitai.com/api/v1/models/{model_id}'
    data = json.loads(response_data)
    response_data = curl_request2(url, cache_dir, bearer_token)
    data = json.loads( response_data )

    if verbose_var:
        parse_json_for_model(data)

    model_name = data['modelVersions'][0]['files'][0]['name']
    model_url = data['modelVersions'][0]['files'][0]['downloadUrl']
    model_trigger = data['modelVersions'][0]['trainedWords']

    if verbose_var:
        print(f'Name/url: {model_name} / {model_url} {image_url}')
        print(f'ImageUrl: {image_url}')
        print(f'Triggers: {model_trigger}')

    return [model_name, model_url, image_url, model_trigger]

def is_valid_directory(directory):
    if os.path.isdir(directory):
        return True
    else:
        return False

if not is_valid_directory(cache_dir):
    os.makedirs(cache_dir, exist_ok=True)

if not is_valid_directory(temp_dir):
    os.makedirs(temp_dir, exist_ok=True)

sources=[source.strip()for source in model_ids.split(',')]

%cd $temp_dir

for model_id in sources:
    clear_output(); print(f'Downloading LORA: {model_id}')

    model_name, model_url, image_url, model_triggers = retrieveDownloadData(model_id, bearer_token, cache_dir)
    model_fn, model_ext = os.path.splitext(model_name)
    if not os.path.isfile(temp_dir + '/' + model_fn + '.jpg') and image_url != '':
        params = f'{image_url} --content-disposition -O {model_fn}.jpg'
        !wget $params

    if not os.path.isfile(temp_dir + '/' + model_name):
        !wget $model_url --content-disposition

convert_images_to_png(temp_dir)
print('Done.')
